## Extract info from Video 

In [1]:
!pip install --upgrade google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 13.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.69.0
    Uninstalling google-cloud-aiplatform-1.69.0:
      Successfully uninstalled google-cloud-aiplatform-1.69.0


In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting
from typing import List, Optional

from dotenv import load_dotenv
load_dotenv()

import os 
GCLOUD_PROJECT = os.getenv("GCLOUD_PROJECT")
GCLOUD_REGION = os.getenv("GCLOUD_REGION")


generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}
safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]


def generate(**kwargs):
    vertexai.init(project=GCLOUD_PROJECT, location=GCLOUD_REGION)
    model = GenerativeModel(
        "gemini-1.5-flash-002",
    )
    responses = model.generate_content(
        [*kwargs],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    for response in responses:
        print(response.text, end="")
        
    return responses
        
def extract_timestamps(video_path: str): 
    video = Part.from_data(mime_type="video/mp4", data=base64.b64encode(video_path))

    text = """Chapterize the video content by grouping the video content into chapters and providing a summary for each chapter. Capture key events, highlights and specific conditions you think are important. If you are not sure about any info, please do not make it up. Return the result in the JSON format with keys as follows : \"timecode\", \"chapterSummary\""""
    generate(video=video, text=text)

generate()